#### Primero necesitamos crear los datos para RAG
Estos datos se incluyen en la carpeta Splits/rag

In [1]:
import json
import csv
import os

In [2]:

def get_file_2(path_file):  
  #print(path_file)
  with open(path_file) as f:
      data = json.load(f)
  #print(data.keys())
  return {key: data[key] for key in data.keys() if key != 'ground_truth' },data['ground_truth']

def get_class(train_path):
  print(train_path)
  if os.path.exists(train_path) and os.path.isdir(train_path):
    carpetas = [nombre for nombre in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, nombre))]
    return carpetas
  else:
    print("El directorio especificado no existe o no es un directorio válido.")

def get_archives(train_path):
  if os.path.exists(train_path) and os.path.isdir(train_path):
    carpetas = [os.path.join(train_path,nombre) for nombre in os.listdir(train_path) if os.path.isfile(os.path.join(train_path, nombre))]
    return carpetas
  else:
    print("El directorio especificado no existe o no es un directorio válido.")

def get_all_files(path_data,max_cont=-1):
    #'./data/splits/train_json'
    classes=get_class(path_data)
    archiver_per_clases={}
    prompt_per_clases={}
    promt_all_reson=[]
    archiver_all_reson=[]
    if len(classes)>0:
      for class_name in classes:
        #print(get_archives(os.path.join(path_data,i)))
        archiver_per_clases[class_name]=get_archives(os.path.join(path_data,class_name))[0:max_cont]
        prompt_per_clases[class_name]=[get_file_2(archive) for archive in  archiver_per_clases[class_name]]
        promt_all_reson.extend(prompt_per_clases[class_name])
        archiver_all_reson.extend(archiver_per_clases[class_name])

        #print(len(archiver_per_clases[class_name]),promt_all_reson)
    else:
       archiver_per_clases['test']=get_archives(os.path.join(path_data,class_name))
       prompt_per_clases['test']=[get_file_2(archive) for archive  in  archiver_per_clases['test']]

    return archiver_per_clases,prompt_per_clases,promt_all_reson,archiver_all_reson

def creating_RAG_csv(text_traindata, csv_filename):
    # Suponiendo que text_traindata es una lista de tuplas donde cada tupla es (texto, label)
    # y el texto contiene el título, el enlace y el cuerpo separados de alguna manera definida

    with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Escribimos la cabecera del CSV
        writer.writerow(['title', 'body','links','label'])

        for text, label in text_traindata:
            title,body, link = text.values()

            # Escribimos la fila al CSV
            writer.writerow([title, body,link,label])

rag_data_csv='rag_data.csv'


In [3]:
import pandas as pd
data=pd.read_csv(rag_data_csv)
display(data)

title  \
0                                         Werner Vogels   
1                                         Jason Collins   
2                               \nJussi Karlgren, NYU\n   
3                           Ashutosh Dutta's Home Page    
4                                        Document moved   
...                                                 ...   
6611                                    A348 - Top page   
6612                         CS 404 Syllabus - Fall '94   
6613                     Computer Science 635 Home Page   
6614                                             CS 515   
6615  \nBU CLA CS 530: Analysis of Algorithms---Home...   

                                                   body  \
0     If we want to take our distributed systems to ...   
1     Date: Tue, 26 Nov 1996 19:24:33 GMT Server: NC...   
2     Background and Reseach Goals I received my B A...   
3     Date: Tue, 26 Nov 1996 20:25:51 GMT Server: Ap...   
4     WWW Alert:  Redirection response from server i...   
...                                                 ...   
6611  Topics to be covered include: web server admin...   
6612  If you find it necessary to be absent, be sure...   
6613  Date: Tuesday, 26-Nov-96 19:07:35 GMT Server: ...   
6614  CS 515 is the practicum for those students who...   
6615         Coordinates: kfoury@cs , MCS-270, x3-8911.   

                                                  links   label  
0     ['mailto:vogels@cs.cornell.edu', 'http://www.c...   staff  
1     ['http://www.usask.ca', 'http://www.cs.usask.c...   staff  
2     ['http://www.sics.se/', 'http://www.sunet.se:8...   staff  
3     ['http://www.cs.columbia.edu/cgi-bin/finger?du...   staff  
4                 ['http://www.tc.cornell.edu/~bruce/']   staff  
...                                                 ...     ...  
6611  ['http://www.cs.indiana.edu/l/www/home-page.ht...  course  
6612  ['http://www.cs.byu.edu/byu.html', 'http://www...  course  
6613  ['http://www.dcs.uky.edu/~seales/cs635/fall96/...  course  
6614  ['http://simon.cs.cornell.edu/Info/Courses/Spr...  course  
6615  ['http://web.bu.edu/pagetwo.html', 'http://cs-...  course  

[6616 rows x 4 columns]

#### Ahora necesitamos crear un RAG para los datos

In [4]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import os
import pinecone
import time
embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

# get API key from app.pinecone.io and environment from console
PINECONE_API_KEY=''#'bc795b6a-d0aa-4877-bdc6-414135eb4bef'
PINECONE_ENV='gcp-starter'
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY') or PINECONE_API_KEY,
    environment=os.environ.get('PINECONE_ENVIRONMENT') or PINECONE_ENV
)


docs = [
    "this is one document",
    "and another document"
]

embeddings = embed_model.embed_documents(docs)

print(f"We have {len(embeddings)} doc embeddings, each with "
      f"a dimensionality of {len(embeddings[0])}.")

/home/alourido/anaconda3/envs/alpaca/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


We have 2 doc embeddings, each with a dimensionality of 384.


In [5]:
index_name = 'llama-2-rag'
#index_name = 'llama-2-rag-2'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=len(embeddings[0]),
        metric='cosine'
    )
    # wait for index to finish initialization
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)
else:
    pinecone.delete_index(index_name)
    pinecone.create_index(
        index_name,
        dimension=len(embeddings[0]),
        metric='cosine'
    )
    # wait for index to finish initialization
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)
index = pinecone.Index(index_name)
index.describe_index_stats()
pinecone.list_indexes()

['llama-2-rag']

In [6]:
import sys
batch_size = 16

for i in range(0, len(data), batch_size):
    i_end = min(len(data), i+batch_size)
    batch = data.iloc[i:i_end]
    ids = ids = [str(index) for index in range(i,i_end)]
    #texts = [x['body'] for i, x in batch.iterrows()]
    #'{'Title': 'CSc425/525 Home Page', 'Body': , 'link': ['http://www.cs.arizona.edu/classes/cs452', 'http://www.cs.arizona.edu/people/llp', 'mailto:llp@cs']}' 
    texts = [f"Text: Title:{x['title']} Body: {x['body']} link: {x['links']}  Classification: {x['label']}" for i, x in batch.iterrows()]
    embeds = embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
    {   'text': f"Text: Title:{x['title']} Body: {x['body']} link: {x['links']}  Classification: {x['label']}",
        'body': '' if pd.isna(x['body']) else x['body'],
        'links': '' if pd.isna(x['links']) else x['links'],
        'title': '' if pd.isna(x['title']) else x['title'],
        'label': '' if pd.isna(x['label']) else x['label']
        } for i, x in batch.iterrows()
    ]
    
    try:
        index.upsert(vectors=zip(ids, embeds, metadata))
    except:
        # add to Pinecone
        sice=sys.getsizeof(zip(ids, embeds, metadata))
        sice2=sys.getsizeof(embeds)
        print(sice,sice2)
        print(zip(ids, embeds, metadata))

64 184


In [7]:
from langchain.vectorstores import Pinecone

text_field = 'text'  # field in metadata that contains text content

vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

/home/alourido/anaconda3/envs/alpaca/lib/python3.11/site-packages/langchain/vectorstores/pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [8]:
query='text'
vectorstore.similarity_search(
    query,  # the search query
    k=5  # returns top 3 most relevant chunks of text
)

[Document(page_content="Text: Title:P. K. Nettle Body: If you don't have one of those, send me mail. link: ['http://www.cs.utexas.edu/users/pkn/images/stupid.gif', 'http://www.cs.utexas.edu/users/pkn/tex.html', 'http://www.cs.utexas.edu/users/pkn/ibm.html']  Classification: staff", metadata={'body': "If you don't have one of those, send me mail.", 'label': 'staff', 'links': "['http://www.cs.utexas.edu/users/pkn/images/stupid.gif', 'http://www.cs.utexas.edu/users/pkn/tex.html', 'http://www.cs.utexas.edu/users/pkn/ibm.html']", 'title': 'P. K. Nettle'}),
 Document(page_content="Text: Title:Communicating Body: My PGP key . link: ['//www.cs.indiana.edu/finger/cs.washington.edu/bam', '//www.cs.indiana.edu/finger/cs.washington.edu/bam', 'mailto:bam@cs.washington.edu']  Classification: other", metadata={'body': 'My PGP key .', 'label': 'other', 'links': "['//www.cs.indiana.edu/finger/cs.washington.edu/bam', '//www.cs.indiana.edu/finger/cs.washington.edu/bam', 'mailto:bam@cs.washington.edu']", 

In [10]:
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any
from llama_cpp import Llama
class LlamaLLM(LLM):
    model_path: str
    llm: Llama

    @property
    def _llm_type(self) -> str:
        return "llama-cpp-python"

    def __init__(self, model_path: str, **kwargs: Any):
        model_path = model_path
        llm = Llama(model_path=model_path,**kwargs)
        super().__init__(model_path=model_path, llm=llm, **kwargs)

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = self.llm(prompt, stop=stop or [])
        return response["choices"][0]["text"]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"model_path": self.model_path}
        
model_path = "../llama_models/llama2-chat-ayb-13b.Q5_K_M.gguf"
llm = LlamaLLM(model_path=model_path,n_ctx=4096)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from ../llama_models/llama2-chat-ayb-13b.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q5_K     [  5120,  5120,    

In [18]:
def creating_promt_2(class_name,text):
  class_name=(', ').join(class_name)
  return f"Classify the text in this class : [{class_name}]. Reply with only one of these words: [{class_name}]. \n\
  Text: '{text}' \n\
  Classication: "


#train,promt_train,b,d
path_data='../data/splits/train_new_short'
classes=get_class(path_data)
query=creating_promt_2(classes,d[0])
print(query)

../data/splits/train_new_short
Classify the text in this class : [staff, project, other, faculty, department, student, course]. Reply with only one of these words: [staff, project, other, faculty, department, student, course]. 
  Text: '../data/splits/train_new_short/staff/brryz.json' 
  Classication: 


#### Y solo quedaria ejecutar el modelo 13b

In [ ]:
llm(query)

In [ ]:
from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=vectorstore.as_retriever()
)

In [ ]:
rag_pipeline(query)